<a href="https://colab.research.google.com/github/skagnihotri/urban_sound_classification/blob/master/Urban_sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#dataset downloaded
!wget -c https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O - | sudo tar -xz

--2019-08-15 18:30:17--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   5.61G  59.4MB/s    in 1m 54s  

2019-08-15 18:32:12 (50.3 MB/s) - written to stdout [6023741708/6023741708]



In [0]:
!apt-get install ffmpeg #backend for librosa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [0]:
pip install librosa #sound preprocessing lib

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
%matplotlib inline

In [0]:
data = pd.read_csv("/content/UrbanSound8K/metadata/UrbanSound8K.csv")

In [0]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [0]:
#count of datapoints in each of the folders
data["fold"].value_counts()

4     990
5     936
3     925
2     888
1     873
7     838
10    837
6     823
9     816
8     806
Name: fold, dtype: int64

In [0]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(data)):
    path = f"/content/UrbanSound8K/audio/fold{data.fold[i]}/{data.slice_file_name[i]}"
    y, sr = librosa.load(path= path)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    if data.fold[i] != 10:
        X_train.append(features)
        y_train.append(data["class"][i])
    else :
        X_test.append(features)
        y_test.append(data["class"][i])
    print(i, end=" ")

TypeError: ignored

In [0]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9, 40, 5), (1, 40, 5), (9,), (1,))

In [0]:
# saving training and testing csv files
